<a href="https://colab.research.google.com/github/Sil477/Politecnico/blob/main/HugginFace_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1 - Pipeline

In [1]:
!pip install transformers -q

[Librería Transformers](https://github.com/huggingface/transformers)

In [3]:
from transformers import pipeline

In [4]:
# tarea de calsficiación
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [13]:
res = classifier("Me encantan las clases de Nechu, explica genial")
print(res)

[{'label': 'POSITIVE', 'score': 0.8972371220588684}]


In [14]:
res = classifier("El profesor es bueno todo será sencillo")
print(res)

[{'label': 'NEGATIVE', 'score': 0.6551787257194519}]


#### Selección del modelo

In [17]:
# seleccionamos el mismo modelo que tenemos por defecto
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

In [18]:
res = classifier("El profesor es bueno todo será sencillo")
print(res)

[{'label': 'NEGATIVE', 'score': 0.6551787257194519}]


Existe una amplia variedad de 'pipelines': [lista de pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline)

### 2 - Modelo y Tokenizer

El modelo lo hemos ejecutado con apenas una línea, pero realmente hay bastantes etapas que ocurren por debajo. En el siguiente código vamos a ver las más importantes.

In [19]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [24]:
# Primero veamos cuales son las etapas anteriores con el mismo modelo
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
#model_name = "pysentimiento/robertuito-sentiment-analysis"

In [25]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer
)

res = classifier("El profesor es bueno todo será sencillo")
print(res)

[{'label': 'NEGATIVE', 'score': 0.6551787257194519}]


### 3 - ¿Para qué sirve el tokenizer?

Codificación (antes del LLM)

In [26]:
secuencia = "El profesor es bueno todo será sencillo"
res = tokenizer(secuencia)
print(res)

{'input_ids': [101, 3449, 11268, 2229, 2953, 9686, 20934, 16515, 28681, 2080, 26358, 12411, 6895, 7174, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Paso a paso

In [27]:
tokens = tokenizer.tokenize(secuencia)
print(tokens)

['el', 'prof', '##es', '##or', 'es', 'bu', '##eno', 'tod', '##o', 'sera', 'sen', '##ci', '##llo']


In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[459, 5934, 442, 1220, 658, 1504, 9764]


Decodificar (después del LLM

In [ ]:
tokenizer.decode(res['input_ids'])

'<s> el profesor es bueno todo será sencillo</s>'

### 4 - Guardar modelo y tokenizer en local

In [ ]:
model_path = ("./modelo")
tokenizer.save_pretrained(model_path)
model.save_pretrained(model_path)

In [ ]:
tokenizer_local = AutoTokenizer.from_pretrained(model_path)
model_local = AutoModelForSequenceClassification.from_pretrained(model_path)

### 5 - Pytorch

También compatible con tensorflow

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
sentences = [
    "He estado deseando un curso así toda mi vida",
    "Me encanta HuggingFace"
]

Tokenizer

In [ ]:
batch = tokenizer(
    sentences,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)
print(batch)

Modelo

In [ ]:
with torch.no_grad():
    outputs = model(**batch)
    predictions = F.softmax(outputs.logits, dim=1)
    labels = torch.argmax(predictions, dim=1)

In [ ]:
# Logits
print(outputs)

In [ ]:
# Neg, Neu, Pos
print(predictions)

In [ ]:
# etiquetas
print(labels)

In [ ]:
model